<pre>
Name: Ritika Sarkar
Reg. No.: 19BAI1031
</pre>

#### C. Multiclass classification on [Face expression Dataset](https://www.kaggle.com/jonathanoheix/face-expression-recognition-dataset)

In [2]:
### on kaggle
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import random
# from google.colab.patches import cv2_imshow
import pandas as pd 
import os
from skimage.transform import resize
from skimage.io import imread

In [7]:
classes = ['angry','disgust','fear','happy','neutral','sad','surprise']

In [8]:
flat_data_arr = [] #input array
target_arr = [] #output array
datadir = '../input/face-expression-recognition-dataset/images/train'

#path which contains all the categories of images
for i in classes:
    print(f'loading... category : {i}')
    path = os.path.join(datadir,i)
    pa = os.listdir(path)
    for img in pa[:400]: # taking only a subset of images as RAM becomes full otherwise
        img_array = imread(os.path.join(path,img))
        img_resized = resize(img_array,(150,150,3))
        flat_data_arr.append(img_resized.flatten())
        target_arr.append(classes.index(i))
    print(f'loaded category:{i} successfully')

loading... category : angry
loaded category:angry successfully
loading... category : disgust
loaded category:disgust successfully
loading... category : fear
loaded category:fear successfully
loading... category : happy
loaded category:happy successfully
loading... category : neutral
loaded category:neutral successfully
loading... category : sad
loaded category:sad successfully
loading... category : surprise
loaded category:surprise successfully


In [9]:
flat_data = np.array(flat_data_arr)
target = np.array(target_arr)
dfn = pd.DataFrame(flat_data) #dataframe
dfn['Target'] = target

In [10]:
dfn.tail()

,0,1,2,3,4,5,6,7,8,9,...,67491,67492,67493,67494,67495,67496,67497,67498,67499,Target
2795,0.738348,0.738348,0.738348,0.704692,0.704692,0.704692,0.734141,0.734141,0.734141,0.767798,...,0.834902,0.834902,0.834902,0.842588,0.842588,0.842588,0.833804,0.833804,0.833804,6
2796,0.441757,0.441757,0.441757,0.354918,0.354918,0.354918,0.430902,0.430902,0.430902,0.517741,...,0.266675,0.266675,0.266675,0.282333,0.282333,0.282333,0.264438,0.264438,0.264438,6
2797,0.999120,0.999120,0.999120,0.999948,0.999948,0.999948,0.999224,0.999224,0.999224,0.998395,...,0.636165,0.636165,0.636165,0.639085,0.639085,0.639085,0.635747,0.635747,0.635747,6
2798,0.998292,0.998292,0.998292,0.997464,0.997464,0.997464,0.998188,0.998188,0.998188,0.999016,...,0.998290,0.998290,0.998290,0.998642,0.998642,0.998642,0.998240,0.998240,0.998240,6
2799,0.881613,0.881613,0.881613,0.907112,0.907112,0.907112,0.884800,0.884800,0.884800,0.859300,...,0.759851,0.759851,0.759851,0.759478,0.759478,0.759478,0.759904,0.759904,0.759904,6


In [11]:
x = dfn.iloc[:,:-1] #input data 
y = dfn.iloc[:,-1] #output data

In [12]:
y

0       0
1       0
2       0
3       0
4       0
       ..
2795    6
2796    6
2797    6
2798    6
2799    6
Name: Target, Length: 2800, dtype: int64

In [13]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x, y, test_size = 0.20, random_state = 42)

In [14]:
# non-linear svm
svm = cv2.ml.SVM_create()
svm.setType(cv2.ml.SVM_C_SVC)
svm.setC(0.1)                 # non-linearity
svm.setKernel(cv2.ml.SVM_LINEAR)
svm.setTermCriteria((cv2.TERM_CRITERIA_MAX_ITER, int(1e6), 1e-6))

In [15]:
svm.train(np.asarray(x_train, np.float32), cv2.ml.ROW_SAMPLE, np.array(y_train))

True

In [16]:
response = svm.predict(np.asarray(x_test, np.float32))[1]

In [17]:
sv = svm.getUncompressedSupportVectors() # the support vectors
print(sv)

[[0.20188235 0.20188235 0.20188235 ... 0.56738824 0.56738824 0.56738824]
 [0.5553631  0.5553631  0.5553631  ... 0.30629334 0.30629334 0.30629334]
 [0.00557804 0.00557804 0.00557804 ... 0.5314337  0.5314337  0.5314337 ]
 ...
 [0.03745569 0.03745569 0.03745569 ... 0.6157396  0.6157396  0.6157396 ]
 [0.8970808  0.8970808  0.8970808  ... 0.9194839  0.9194839  0.9194839 ]
 [0.5185192  0.5185192  0.5185192  ... 0.2391906  0.2391906  0.2391906 ]]


In [18]:
# performance metrics
from sklearn.metrics import accuracy_score
print("Accuracy score", accuracy_score(y_test, [x[0] for x in response]))

Accuracy score 0.2910714285714286


In [19]:
from sklearn.metrics import confusion_matrix
print("Confusion matrix\n", confusion_matrix(y_test, [x[0] for x in response]))

Confusion matrix
 [[21 14 12 11  9  4  5]
 [15 37 11 10  5  6  5]
 [15  9 17  6  7  9 14]
 [17 13 16 27  7  3  3]
 [14  9  9 11 18 10  5]
 [12 11 15  8 18 15  4]
 [ 4  8 14  8  6  5 28]]


In [20]:
from sklearn.metrics import classification_report
print(classification_report(y_test,[x[0] for x in response]))

              precision    recall  f1-score   support

           0       0.21      0.28      0.24        76
           1       0.37      0.42      0.39        89
           2       0.18      0.22      0.20        77
           3       0.33      0.31      0.32        86
           4       0.26      0.24      0.25        76
           5       0.29      0.18      0.22        83
           6       0.44      0.38      0.41        73

    accuracy                           0.29       560
   macro avg       0.30      0.29      0.29       560
weighted avg       0.30      0.29      0.29       560



#### Observation
1. It is observed that the test accuracy is 29% when we use 400 images from each class of emotion out of 3000s to 4000s, due to RAM constraints. Also as SVM needs more number of iterations i.e., 1e6 iterations to fit on this data, it takes a longer time to run. We have flattened out the 2D image array to 1D and fed them as features to the SVM. The value of C which controls the margin size is relatively large, to prevent more misclassification errors.
2. It was also seen when we initially tested the model with 200 images from each class, the accuracy was 25%, which upon increasing to 400 images per class the accuracy is 29%. This clearly demonstrates that if the RAM constraints are combated, SVM can achieve a competitive accuracy on this dataset.
3. We observe that the precision and recall of the 1st (disgust) and 6th (surprise) classes are highest out of the other classes, which shows that svm was able to learn their feature better despite the less number of image samples given for training.

References:
1. https://medium.com/analytics-vidhya/image-classification-using-machine-learning-support-vector-machine-svm-dc7a0ec92e01
2. https://docs.opencv.org/4.x/d0/dcc/tutorial_non_linear_svms.html